## Install GPU Version 

+ https://anaconda.org/search?q=faiss-gpu
+ Find cmd as "conda install conda-forge::faiss-gpu"
+ Install
+ Restart jupyterlab

In [1]:
import faiss
import numpy as np

### Load Data

In [2]:
d = 64                           # 向量维度
nb = 10000                       # 数据库中向量的数量
np.random.seed(1234)             # 确保结果可重复
xb = np.random.random((nb, d)).astype('float32')  # 数据库向量
xb[:, 0] += np.arange(nb) / 1000.  # 添加一些差异以便于验证

In [5]:
xb.size

640000

### Create Index

#### IndexFlatL2 

In [6]:
index = faiss.IndexFlatL2(d)   # 使用 L2 距离构建索引
print(index.is_trained)        # 输出 'True'，因为 FlatIndex 不需要训练
index.add(xb)                  # 将向量添加到索引
print(index.ntotal)            # 输出 '10000'，表示已添加的向量数量

True
10000


In [ ]:
nq = 10                          # 查询向量的数量
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

k = 4                            # 返回最近邻的数量
D, I = index.search(xq, k)       # 实际进行搜索
print(I)                         # 最近邻的索引
print(D)                         # 对应的距离

#### IVF(Inverted File)

In [9]:
nlist = 100  # 聚类中心的数量
quantizer = faiss.IndexFlatL2(d)  # 使用 L2 距离作为量化器
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

# 训练索引
index_ivf.train(xb)

In [10]:
# 添加向量到索引
index_ivf.add(xb)

# 执行搜索
D, I = index_ivf.search(xq, k)
print(I[-5:])  # 输出最后五个查询的结果
print(D[-5:])  # 输出对应的距离

[[1075  416   34   88]
 [  30  118  479  365]
 [ 364  145 1040 1129]
 [  14   22  261  439]
 [ 782  483  756  918]]
[[7.0537624 7.1303234 7.837903  8.13125  ]
 [6.719946  6.798668  7.099112  7.145878 ]
 [7.5604906 7.5931296 7.912198  8.444745 ]
 [7.436784  7.784588  7.9796133 7.984132 ]
 [6.421344  6.9501457 7.8102565 7.832158 ]]


#### HNSW (Hierarchical Navigable Small World Graphs)

### Search in DB

In [7]:
nq = 10                          # 查询向量的数量
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

k = 4                            # 返回最近邻的数量
D, I = index.search(xq, k)       # 实际进行搜索
print(I)                         # 最近邻的索引
print(D)                         # 对应的距离

[[ 794   72  515 1204]
 [ 393  919  424  923]
 [ 688  806  763  185]
 [ 950  230  317  703]
 [ 199  409  411  794]
 [ 383  227 1075  416]
 [ 630  133   32  917]
 [1407  235  779  470]
 [  14   22  581  507]
 [ 637  782  483  680]]
[[5.9375467 6.2643566 6.2803383 6.396471 ]
 [7.596614  8.063351  8.070611  8.196373 ]
 [7.180752  7.3030562 7.459653  7.502161 ]
 [6.0598154 6.094823  6.198604  6.550714 ]
 [5.482876  7.1775966 7.246011  7.33661  ]
 [6.25152   6.906643  7.0537624 7.1303234]
 [6.4409313 6.578717  6.6783032 6.6917696]
 [7.0216146 7.0476847 7.33723   7.4447927]
 [7.436784  7.784588  7.9513555 7.9681993]
 [5.997012  6.421344  6.9501457 7.008673 ]]
